In [3]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.appName("spark").getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# check the total number of genres and select the represntative genres

In [14]:
import sqlite3
con = sqlite3.connect("../Books.db")
cur = con.cursor()
cur.execute("SELECT genres FROM Foreign_book")
data = cur.fetchall()
print(len(data))
print(type(data))
print(type(data[0]))
print(data[0])

45895
<class 'list'>
<class 'tuple'>
('Young Adult, Fiction, Dystopia, Fantasy, Science Fiction, Romance, Adventure, Teen, Post Apocalyptic, Action',)


In [15]:
rdd = sc.parallelize(data).map(lambda genres: genres[0])
print(rdd.take(1))

['Young Adult, Fiction, Dystopia, Fantasy, Science Fiction, Romance, Adventure, Teen, Post Apocalyptic, Action']


In [16]:
rdd1 = rdd.map(lambda line: tuple(line.split(", ")))
print(rdd1.take(1))
print(rdd1.count())

[('Young Adult', 'Fiction', 'Dystopia', 'Fantasy', 'Science Fiction', 'Romance', 'Adventure', 'Teen', 'Post Apocalyptic', 'Action')]
45895


In [17]:
rdd2 = rdd1.flatMap(lambda x: x)
print(rdd2.take(5))
print(rdd2.count())

['Young Adult', 'Fiction', 'Dystopia', 'Fantasy', 'Science Fiction']
395626


In [18]:
## reduceBy로 하면 계속해서 오류가 나서 python으로 변환하여 실행함
genre_dic = {}

for i, value in enumerate(rdd2.collect()):
    if value in genre_dic:
        genre_dic[value] += 1
    else:
        genre_dic[value] = 1

print("total genres:",len(genre_dic))
it = zip(genre_dic.keys(), genre_dic.values())
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))

total genres: 980
('Young Adult', 11459)
('Fiction', 30705)
('Dystopia', 1637)
('Fantasy', 14528)
('Science Fiction', 5191)


In [19]:
all_genres = dict(sorted(genre_dic.items(), key=lambda x: x[1], reverse=True))
count = 0
for i, item in enumerate(zip(all_genres.keys(), all_genres.values())):
    if count >= 25:
        break
    print(i,item)
    count+=1

# all_genres = {"name":"hae", "age":12}
# print(all_genres.items())
# print(all_genres)

0 ('Fiction', 30705)
1 ('Romance', 14837)
2 ('Fantasy', 14528)
3 ('Young Adult', 11459)
4 ('Contemporary', 10156)
5 ('Adult', 8034)
6 ('Nonfiction', 8020)
7 ('Novels', 7608)
8 ('Mystery', 7514)
9 ('Historical Fiction', 7476)
10 ('Audiobook', 7157)
11 ('Classics', 6703)
12 ('Adventure', 6282)
13 ('Historical', 6231)
14 ('Paranormal', 5804)
15 ('Literature', 5709)
16 ('Science Fiction', 5191)
17 ('Childrens', 5052)
18 ('Thriller', 4483)
19 ('Magic', 4112)
20 ('Humor', 4094)
21 ('Crime', 3569)
22 ('History', 3553)
23 ('Contemporary Romance', 3411)
24 ('Urban Fantasy', 3368)


In [20]:
all_genres = list(all_genres.keys())
print(len(all_genres))
with open("../data/foreign-all-genres.txt", "wb") as f:
    for value in all_genres:
        f.write((value+"\n").encode("UTF-8"))

980


In [21]:
exclued_list = [6, 10]
selected_genres = []
for i, genre in enumerate(all_genres):
    if i==25:
        break
    if i in exclued_list:
        pass
    else:
        selected_genres.append(genre)

print(len(selected_genres))
print((selected_genres))

with open("../data/foreign-representative-genres.txt", "wb") as f:
    for value in selected_genres:
        f.write((value+"\n").encode("UTF-8"))

23
['Fiction', 'Romance', 'Fantasy', 'Young Adult', 'Contemporary', 'Adult', 'Novels', 'Mystery', 'Historical Fiction', 'Classics', 'Adventure', 'Historical', 'Paranormal', 'Literature', 'Science Fiction', 'Childrens', 'Thriller', 'Magic', 'Humor', 'Crime', 'History', 'Contemporary Romance', 'Urban Fantasy']


In [22]:
others_genres = []
new_exclued_list = [ i for i in range(25) if i not in exclued_list ]
for i, genre in enumerate(all_genres):
    if i in new_exclued_list:
        pass
    else:
        others_genres.append(genre)

print(len(others_genres)+len(selected_genres))
print(len(others_genres))

with open("../data/foreign-others-genres.txt", "wb") as f:
    for value in others_genres:
        f.write((value+"\n").encode("UTF-8"))

980
957


In [23]:
with open("../data/foreign-all-genres.txt", "rb") as f:
    genre = f.read().decode("UTF-8").split("\n")
    genre.remove("")    

In [24]:
print(len(genre))
print(type(genre[0]))
genres_dic = {}
for i in range(len(genre)):
    genres_dic[genre[i]] = i+1
print(len(genres_dic))
print((genres_dic)["Fantasy"])

980
<class 'str'>
980
3


In [25]:
import pickle
with open("../data/foregin-all-genres-dic.pkl", "wb") as f:
    pickle.dump(genres_dic, f)

# data cleaning

In [6]:

import sqlite3
from flask import Flask, request, jsonify

conn = sqlite3.connect("../Books.db")
cur = conn.cursor()
# cur.execute("SELECT title, author, publisher FROM Foreign_book WHERE title LIKE \"%헤%드%스%파%\"")
cur.execute("SELECT id FROM Foreign_book WHERE pages==\'1 page\'")
# cur.execute("DELETE FROM Foreign_book WHERE id==122")
data = cur.fetchall()
print(len(data))
conn.commit()

# for id in data:
#     cur.execute(f"DELETE FROM Foreign_book WHERE id=={id[0]}")
#     conn.commit()

cur.close()
conn.close()

0


# practice

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

In [22]:
genres_list = (data["genres"].iloc[:].tolist())

In [27]:
# import sqlite3
# conn = sqlite3.connect("../Books.db")
# cur = conn.cursor()
# cur.execute("SELECT genres FROM Foreign_book")
# data = cur.fetchall()
# print(data)
# conn.commit()
# conn.close()

In [23]:
import numpy as np
x = np.array([10,0,1,1])
y = np.array([9,1,0,0])
z = np.array([3,0,1,1])
from scipy.spatial import distance
print(distance.cosine(x, y))
print(distance.cosine(x, x))
print(distance.cosine(x, z))
print(distance.cosine(y, z))

0.01590846210843544
0.0
0.04466970555754857
0.10099833641376377


In [28]:
import math
def consine_similarity(x, y):
    return x.dot(y)/(np.sqrt((x**2).sum())*np.sqrt((y**2).sum()))

print(consine_similarity(x,y))
print(consine_similarity(x,z))
# print(consine_similarity(y,z))


0.9840915378915646
0.9553302944424514


In [35]:
dept = [("Finance",10),("Marketing",20),("Sales",30),("IT",40)]
temprdd = sc.parallelize(dept)
df = temprdd.toDF()
df.show()

+---------+---+
|       _1| _2|
+---------+---+
|  Finance| 10|
|Marketing| 20|
|    Sales| 30|
|       IT| 40|
+---------+---+



In [6]:
import pandas as pd
data = pd.read_csv("./data/foreign-books.csv")
title = data["title"].iloc[0]
genres = data["genres"].iloc[0].replace("[","").replace("]","").replace("\'","")
author = data["author"].iloc[0]
rating = data["rating"].iloc[0]
publisher = data["publisher"].iloc[0]
p_date = data["publishDate"].iloc[0]
pages = data["pages"].iloc[0]
language = data["language"].iloc[0]
description = data["description"][0]
imgUrl = data["coverImg"][0]

print(title)
print(genres)
print(author)
print(rating)
print(publisher)
print(p_date)
print(pages)
print(language)
print(description)
print(imgUrl)
print(len(data))

The Hunger Games
Young Adult, Fiction, Dystopia, Fantasy, Science Fiction, Romance, Adventure, Teen, Post Apocalyptic, Action
Suzanne Collins
4.33
Scholastic Press
09/14/08
374
English
WINNING MEANS FAME AND FORTUNE.LOSING MEANS CERTAIN DEATH.THE HUNGER GAMES HAVE BEGUN. . . .In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and once girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV.Sixteen-year-old Katniss Everdeen regards it as a death sentence when she steps forward to take her sister's place in the Games. But Katniss has been close to dead before—and survival, for her, is second nature. Without really meaning to, she becomes a contender. But if she is to win, she will have to start making choices that weight survival against human

# data cleaning

In [26]:
import sqlite3
con = sqlite3.connect("../Books.db")
cur = con.cursor()
# cur.execute("DELETE FROM Foreign_book Where genres==\'\'")
cur.execute("SELECT * FROM Foreign_book Where genres==\'\'")
data = cur.fetchall()
con.commit()
cur.close()
con.close()
print(len(data))

0
